# Chapter 10 - Using the Subclassing, Saving Models and Using Callbacks

With keras, you can subclass the ```model``` class to create models with more flexibility from a given framework. 

In [1]:
import tensorflow as tf
from tensorflow import keras 

In [2]:
class WideAndDeepModel(keras.Model):
    def __init___(self, units=30, activation='relu', **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(units, activation)
        self.hidden2 = keras.layers.Dense(units, activation)
        self.main_output = keras.layers.Dense(1)
        self.aux_output = keras.layers.Dense(1)

    def call(self, inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.concatenate([input_A, input_B])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output, aux_output

In [3]:
model = WideAndDeepModel()

Same model from the end of Example2, except we do not need to create all the inputs. We just input the argument to the ```call()``` method, and separate the creation of the layers in the constructor from their usage in the ```call()``` method. You can do pretty much anything within the ```call()``` method, this construction of the model gives much more adaptability. 

This flexibility is met with some problems.
- Keras can not easily inspect this model since the archetecture is hidden within the call method, so it can not be easily saved or cloned
- The summary method does not give any information on how the layers are connected to each other
- Keras also cannot check types and shapes ahead of time, so it is easier to make mistakes

Saving and Resotring a Keras model using the Sequential or Functional API:

(this will not work when subclassing)

In [ ]:
# define, compile, and fit the model as usual ... 
# saving
model.save('my_model.h5')
# loading
model = keras.model.load_model('my_model.h5')

Saving the model is useful as training will often last several hours, so it prevents accidentally deleting the model and having to retrain all over again. To avoid losing everything if the computer crashes during training, use callbacks. These can also be used to stop training early if a predetermined level of accuracy is reached. 
You can also set ```save_best_only``` to true in the checkpoint constructor to save only the best model on the validation set.

In [ ]:
# will not compile, but shows how to use a callback 
checkpoint_cb = keras.callbacks.ModelCheckpoint('my_model.h5', save_best_only=True)
history = model.fit(X_train, y_train, epochs=10, callbacks=[checkpoint_cb])


Can also use the ```EarlyStopping``` callback to interrput training when no progress is being made on the validation set for a number of epochs, defined by the ```patience``` argument. This callback will keep track of the best weights and restore them for you at the end of training.

In [ ]:
early_stopping_callback = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)